In [1]:
from openslide import OpenSlide, lowlevel 
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import numpy as np
from PIL import Image, ImageDraw
import math
from matplotlib import cm
import staintools
import glob
import os
from tqdm import tqdm
import cv2 as cv
import multiprocessing
import xml.etree.ElementTree as ET

Image.MAX_IMAGE_PIXELS = 325755924

In [2]:
def normalizeWSI(slides, idx, method):
    for i in idx:
        # --- Process per WSI --- #
        slide_name = slides[i]
        name_wo_ndpi = slide_name.split('/')[-1].split('.ndpi')[0]
        print(f'[INFO] Processing slide {name_wo_ndpi}')

        folder_patches = os.path.join(root_dir, f'scanner_old/{name_wo_ndpi}/patches')

        if method == 'vahadane':
            output_folder = os.path.join(root_dir, f'scanner_old/{name_wo_ndpi}/vahadane')
        if method == 'macenko':
            output_folder = os.path.join(root_dir, f'scanner_old/{name_wo_ndpi}/macenko')

        # print(f'    Reading patches from:   {folder_patches}')
        # print(f'    Saving Vahadane to:     {output_folder_vahadane}')

        os.makedirs(output_folder, exist_ok=True)

        for img in glob.glob(os.path.join(folder_patches,'*.png')):
            source = staintools.read_image(img)

            norm_img = normalize_alternative(source, target, method, True)
            output_filename = os.path.join(output_folder, img.split('/')[-1])
            norm_img = Image.fromarray(norm_img)
            norm_img.save(output_filename)


def normalize_alternative(image, ref, method, standartize_brightness):
    if standartize_brightness:
        image = staintools.LuminosityStandardizer.standardize(image)
        ref = staintools.LuminosityStandardizer.standardize(ref)
            
    if method == 'reinhard':
        normalizer = staintools.ReinhardColorNormalizer()
    if method == 'vahadane':
        normalizer = staintools.StainNormalizer(method='vahadane')
    if method == 'macenko':
        normalizer = staintools.StainNormalizer(method='macenko')
            
    normalizer.fit(ref)
    norm_img = normalizer.transform(image)
        
    return norm_img  

In [10]:
NUM_PROC = 4 # number of processors in PC

# Define the path for the slides & labels
slides_path = '/home/gabriel/Documents/phd/StratifiAD_project/AT8Dataset/AT8_wsi/'
slides = sorted(glob.glob(os.path.join(slides_path,'*.ndpi')))

patchSize = 128
root_dir = '/home/gabriel/Documents/phd/StratifiAD_project/dataset'
target = staintools.read_image(f"../data/norm_reference_{patchSize}x{patchSize}_oldscanner.png")

jobs = []
idx = [5, 6, 7, 9] # WSIs to be processed

for i in range(NUM_PROC):
    imgs_per_proc = (len(idx)//NUM_PROC)
    print(f'Processor {i}: WSIs {idx[i*imgs_per_proc:(i+1)*imgs_per_proc]}')
    process = multiprocessing.Process(target=normalizeWSI, args=(slides, idx[i*imgs_per_proc:(i+1)*imgs_per_proc], 'macenko'))
    jobs.append(process)

for j in jobs:
    j.start()

for j in jobs:
    j.join()

Processor 0: WSIs [5]
Processor 1: WSIs [6]
Processor 2: WSIs [7]
Processor 3: WSIs [9]
[INFO] Processing slide APP-point-mutation_CS427
[INFO] Processing slide APP-point-mutation_P29-11
[INFO] Processing slide APP_point-mutation_BBN-9608
[INFO] Processing slide sAD_5138
